In [16]:
import requests, zipfile, io
import pandas as pd
import numpy as np
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import colorcet as cc
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_log_error, mean_squared_error

In [2]:
bpl.output_notebook()

Loading BokehJS ...

Definición del modelo de CDMX, los detalles del modelo se encuentran en [https://modelo.covid19.cdmx.gob.mx/modelo-epidemico](https://modelo.covid19.cdmx.gob.mx/modelo-epidemico)

In [3]:
# Susceptibles equation
def dS_dt(S, I, R_t, t_inf):
    return -(R_t / t_inf) * I * S

# Expuestos equation
def dE_dt(S, E, I, R_t, t_inf, t_inc):
    return (R_t / t_inf) * I * S - (E / t_inc)

# Infectados equation
def dI_dt(I, E, t_inc, t_inf):
    return (E / t_inc) - (I / t_inf)

#Leves equation
def dL_dt(I, L, p_grave, t_inf, t_rl):
    return (1-p_grave)*(I /t_inf) - (L / t_rl)

#Graves equation
def dG_dt(I, G, p_grave, t_inf, t_hosp):
    return p_grave*(I / t_inf) - (G / t_hosp)

# Hospializados equation
def dH_dt(G, H, t_hosp, p_icu, t_rh, t_icu):
    return (G / t_hosp) - (1-p_icu)*(H / t_rh) - (p_icu)*(H / t_icu)

# ICU equation
def dICU_dt(H, ICU, p_icu, t_icu, p_m, t_ricu, t_m):
    return p_icu*(H / t_icu) - (1-p_m)*(ICU / t_ricu) - p_m*(ICU / t_m)

# Recovered equation
def dR_dt(L, H, ICU, t_rl, p_icu, t_rh, p_m, t_ricu):
    return (L / t_rl) + (1-p_icu)*(H / t_rh) + (1-p_m)*(ICU / t_ricu)

# Deaths equation
def dD_dt(ICU, t_m, p_m):
    return p_m*(ICU / t_m)


def CDMX_model(t, y, R_t, t_inf=2.9, t_inc=5.2, t_rh=12, t_rl=14, t_hosp=4, t_icu=1, t_ricu=7, t_m=8, p_m=.03, p_grave=.138, p_icu=0.05):

    if callable(R_t):
        reprod = R_t(t)
    else:
        reprod = R_t
        
    S, E, I, L, G, H, ICU, R, D = y
    
    S_out = dS_dt(S, I, reprod, t_inf)
    E_out = dE_dt(S, E, I, reprod, t_inf, t_inc)
    I_out = dI_dt(I, E, t_inc, t_inf)
    L_out = dL_dt(I, L, p_grave, t_inf, t_rl)
    G_out = dG_dt(I, G, p_grave, t_inf, t_hosp)
    H_out = dH_dt(G, H, t_hosp, p_icu, t_rh, t_icu)
    ICU_out = dICU_dt(H, ICU, p_icu, t_icu, p_m, t_ricu, t_m)
    R_out = dR_dt(L, H, ICU, t_rl, p_icu, t_rh, p_m, t_ricu)
    D_out = dD_dt(ICU, t_m, p_m)
    return [S_out, E_out, I_out, L_out, G_out, H_out, ICU_out, R_out, D_out]

Ejecución del modelo de CDMX con los parámetros que reportan, hay dos instancias, una sin intervenciones y la otra con 3 intervenciones

In [213]:
N = 22000000
n_infected = 40*2
max_days = 200
inicio = '2020-03-10'
tiempos = pd.date_range(start=inicio, periods=max_days).values
serie_tiempos = pd.Series(tiempos)

initial_state = [(N - n_infected)/ N, 0, n_infected / N, 0, 0, 0, 0, 0, 0]

R_0 = 2.83
t_inf=2.9
t_inc=5.2
t_rh=12
t_rl=14
t_hosp=4
t_icu=1
t_ricu = 7
t_m=8
p_m=.65
p_grave=0.138
p_icu=0.05

intervenciones = [{"R":2.83,"fecha":inicio},{"R":2.2,"fecha":"2020-03-22"},{"R":0.95,"fecha":"2020-03-31"}]

def rep(t):
    rfinal = 2.83
    for intervencion in intervenciones:
        tint = serie_tiempos[(serie_tiempos-pd.Timestamp(intervencion["fecha"])).dt.days==0].index[0]
        if (t > tint):
            rfinal = intervencion["R"]
    return rfinal

args_sin_interv = (R_0, t_inf, t_inc, t_rh, t_rl, t_hosp, t_icu, t_ricu, t_m, p_m, p_grave, p_icu)
args_con_interv = (rep, t_inf, t_inc, t_rh, t_rl, t_hosp, t_icu, t_ricu, t_m, p_m, p_grave, p_icu)

sol_sin_interv = solve_ivp(CDMX_model, [0, max_days], initial_state, args=args_sin_interv, t_eval=np.arange(max_days))
sol_con_interv = solve_ivp(CDMX_model, [0, max_days], initial_state, args=args_con_interv, t_eval=np.arange(max_days))

In [511]:
labels = ["Susceptibles","Expuestos","Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]
colores = {"Susceptibles":"#969696",
           "Recuperados":"Green",
           "Expuestos":"Pink",
           "Infectados":"Red",
           "Leves":"#807DBA",
           "Graves":"#54278F",
           "Hospitalizados":"#CC4C02",
           "ICUs":"#662506",
           "Defunciones":"#525252"}

In [512]:
solucion_sin = pd.DataFrame(sol_sin_interv.y.T*N,columns=labels,index=sol_sin_interv.t)
solucion_con = pd.DataFrame(sol_con_interv.y.T*N,columns=labels,index=sol_con_interv.t)

In [513]:
solucion_sin["Fecha"] = tiempos
solucion_con["Fecha"] = tiempos

In [514]:
p = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX")

In [515]:
for label in ["ICUs"]:
    #p.line(x=solucion_sin["Fecha"],y=solucion_sin[label]/1000,color=colores[label],line_width=3,legend_label=label,line_alpha=0.5)
    p.line(x=solucion_con["Fecha"],y=solucion_con[label]/1000,color=colores[label],line_width=3,legend_label=label,line_alpha=0.5,line_dash="dotted")

In [516]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Miles de personas'
p.legend.location = "center_right"

In [517]:
bpl.show(p)

Obtención de los últimos datos para México

In [201]:
urlnew = "http://187.191.75.115/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip"
fecha_archivo = pd.Timestamp("2020-05-15")

In [21]:
archivo = fecha_archivo.strftime("%y%m%d")+"COVID19MEXICO.csv"

In [18]:
r = requests.get(urlnew)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path="./datos_federales/")

In [34]:
data = pd.read_csv('./datos_federales/' + archivo,encoding="latin1", low_memory=False)

In [35]:
data["FECHA_ACTUALIZACION"] = pd.to_datetime(data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
data["FECHA_INGRESO"] = pd.to_datetime(data["FECHA_INGRESO"],format="%Y-%m-%d")
data["FECHA_DEF"] = pd.to_datetime(data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
data["FECHA_SINTOMAS"] = pd.to_datetime(data["FECHA_SINTOMAS"],format="%Y-%m-%d")

Filtrado de datos

In [36]:
data_cdmx = data[(data["RESULTADO"]==1)&(data["ENTIDAD_UM"]==9)]

In [67]:
time_index = pd.date_range(data_cdmx["FECHA_SINTOMAS"].min(), fecha_archivo, freq='1D')

In [68]:
casos = data_cdmx.groupby("FECHA_SINTOMAS").size().to_frame("Confirmados").reindex(time_index)

In [70]:
casos["Decesos"] = data_cdmx.groupby("FECHA_DEF").size()

In [81]:
casos["Confirmados_acumulados"] = casos["Confirmados"].fillna(0).cumsum()
casos["Decesos_acumulados"] = casos["Decesos"].fillna(0).cumsum()

In [204]:
casos[40:45]

,Confirmados,Decesos,Confirmados_acumulados,Decesos_acumulados
2020-03-10,26.0,NaN,94.0,0.0
2020-03-11,23.0,NaN,117.0,0.0
2020-03-12,21.0,NaN,138.0,0.0
2020-03-13,36.0,NaN,174.0,0.0
2020-03-14,38.0,NaN,212.0,0.0


Graficado de datos para CDMX

In [158]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=600,x_range=(pd.Timestamp("2020-02-14"),pd.Timestamp("2020-03-30")),y_range=(0,800))

In [159]:
s.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Confirmados_acumulados"],bottom=1)
s.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Decesos_acumulados"],bottom=1,color="red")

GlyphRenderer(id='13400', ...)

In [160]:
bpl.show(s)

Definición de función para evaluar el modelo en función de R_0 y calculando el error a los datos reales. esto es la función que usaremos para ajustar R_0 a los datos. Se puede seguir una estrategia similar si se desean ajustar más parámetros

In [612]:
def evaluador(R_0, casos, N, dias_prediccion=0,ajustar_decesos=True,regresar_solucion=False):
    inicio = casos.index[0]
    n_infected = casos["Confirmados"].fillna(0)[inicio]
    max_days = len(casos) + dias_prediccion
    tiempos = pd.date_range(start=inicio, periods=max_days).values
    serie_tiempos = pd.Series(tiempos)
    
    initial_state = [(N - n_infected)/ N, 0, n_infected / N, 0, 0, 0, 0, 0, 0]

    t_inf=2.9
    t_inc=5.2
    t_rh=12
    t_rl=14
    t_hosp=4
    t_icu=1
    t_ricu = 7
    t_m=8
    p_m=.65
    p_grave=0.138
    p_icu=0.05
    
    args = (R_0, t_inf, t_inc, t_rh, t_rl, t_hosp, t_icu, t_ricu, t_m, p_m, p_grave, p_icu)
    
    sol = solve_ivp(CDMX_model, [0, max_days], initial_state, args=args, t_eval=np.arange(max_days))
    
    labels = ["Susceptibles","Expuestos","Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]
    
    solucion = pd.DataFrame(sol.y.T*N,columns=labels,index=tiempos)
    solucion["Defunciones"] = np.clip(solucion["Defunciones"],0,np.inf)
    solucion["Confirmados_acumulados"] = np.clip(solucion[["Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]].sum(axis=1),0,np.inf)
    
    pesos = pd.Series(1/((casos.index-casos.index[-1]).days*-1+1),index=casos.index)
    #pesos = pd.Series(1,index=casos.index)
    
    error_confirmados = mean_squared_log_error(casos["Confirmados_acumulados"],solucion["Confirmados_acumulados"][casos.index].fillna(0),pesos)
    error_decesos = mean_squared_log_error(casos["Decesos_acumulados"],solucion["Defunciones"][casos.index].fillna(0),pesos)
    
    if ajustar_decesos:
        error_final = np.mean([error_confirmados, error_decesos])
    else:
        error_final = error_confirmados
    
    if regresar_solucion:
        return error_final, solucion
    else:
        return error_final

Ejecución del modelo con la R_0 que reportan en CDMX

In [634]:
fecini = pd.Timestamp("2020-02-17")
fecfin = pd.Timestamp("2020-03-22")
dias_pred = 0
fecpred = fecfin + pd.Timedelta(str(dias_pred) + " days")

In [635]:
casos_filt = casos.loc[fecini:fecfin,:]
casos_pred = casos.loc[fecfin:fecpred,:]

In [636]:
error, solucion = evaluador(2.83,casos_filt,9000000,dias_pred,True,True)

In [637]:
solucion[:3]

,Susceptibles,Expuestos,Infectados,Leves,Graves,Hospitalizados,ICUs,Recuperados,Defunciones,Confirmados_acumulados
2020-02-17,8.999999e+06,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.00000
2020-02-18,8.999998e+06,1.062370,0.624982,0.321630,0.052516,-0.002152,0.000673,0.003002,0.0,1.00065
2020-02-19,8.999994e+06,6.614828,-2.089738,1.758442,0.351611,-0.110103,0.011323,-0.083794,0.0,0.00000


Graficado de la solución, se puede apreciar que el valor de R_0 no refleja bien el crecimiento que tienen los datos reales

In [642]:
p = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX")

In [643]:
p.line(x=casos_filt.index,y=casos_filt["Confirmados_acumulados"],color="#0B9A53",line_width=3,legend_label="Casos a ajustar",line_dash="dashed")
p.line(x=casos_pred.index,y=casos_pred["Confirmados_acumulados"],color="#3999E4",line_width=3,legend_label="Casos a predecir",line_dash="dotted")
p.line(x=solucion.index,y=solucion["Confirmados_acumulados"],color="#9B0E63",line_width=3,legend_label="Casos confirmados, modelo R_0 = " + str(R_0))
for label in ["Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]:
    p.line(x=solucion.index,y=solucion[label],color=colores[label],line_width=3,legend_label=label)

In [644]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Miles de personas'
p.legend.location = "top_left"

In [645]:
bpl.show(p)

Ajuste del modelo para obtener la mejor R_0

In [621]:
ajuste = minimize(evaluador,[2.8],args=(casos_filt,9000000,0,True,False),method='L-BFGS-B')
ajuste

      fun: 0.06182513895894985
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([-6.68701206e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 8
      nit: 2
   status: 0
  success: True
        x: array([3.01739485])

In [622]:
R_f = ajuste.x[0]
R_f

3.0173948475728603

In [623]:
error, solucion = evaluador(R_f,casos_filt,9000000,0,False,True)

In [624]:
error

0.08646100619180984

In [625]:
solucion[:3]

,Susceptibles,Expuestos,Infectados,Leves,Graves,Hospitalizados,ICUs,Recuperados,Defunciones,Confirmados_acumulados
2020-02-17,8.999999e+06,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000
2020-02-18,8.999998e+06,1.168737,0.608123,0.330475,0.054302,-0.002706,0.000706,0.002372,0.0,0.993273
2020-02-19,8.999993e+06,7.335874,-2.288374,1.831644,0.365112,-0.112590,0.011294,-0.087984,0.0,0.000000


Graficación del modelo ajustado

In [630]:
p = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX")

In [631]:
p.line(x=casos_filt.index,y=casos_filt["Confirmados_acumulados"],color="#0B9A53",line_width=3,legend_label="Casos a ajustar",line_dash="dashed")
#p.line(x=casos_pred.index,y=casos_pred["Confirmados_acumulados"],color="#3999E4",line_width=3,legend_label="Casos a predecir",line_dash="dotted")
for label in ["Confirmados_acumulados"]:
    p.line(x=solucion.index,y=solucion[label],color="#9B0E63",line_width=3,legend_label="Casos confirmados, modelo R_0 = " + str(R_f))

In [632]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Miles de personas'
p.legend.location = "top_left"

In [633]:
bpl.show(p)

Se puede apreciar que el ajuste es bastánte malo